<h1><center>Dementia Disease Prediction - 2 </center></h1>

## Reading and understanding the Data

In [3]:
%pip install wordcloud

import re
import nltk
from nltk.corpus import stopwords
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score

# Additional libraries for enhanced text analysis and visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

nltk.download("stopwords")
print("Enhanced text analysis libraries loaded successfully!")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/511.1 KB ? eta -:--:--  Downloading wordcloud-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 KB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 KB 1.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Enhanced text analysis libraries loaded successfully!
Enhanced text analysis libraries loaded successfully!


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wathsalya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load dataset
file_path = "dementia_dataset_4.csv"
df = pd.read_csv(file_path)

df.head()

,recordID,Transcript_CTD,Transcript_PFT,Transcript_SFT,Class_label
0,0,NaN,"Pat: People, partner, plate, platter, pants, p...","Pat: (3 seconds) Giraffe, kangaroo, lion, tige...",1
1,1,Pat: (4 seconds) There’s a lad stood on the st...,"Pat: (1 second) Pipe, plane, people (5 seconds...","Pat: (1 second) Dogs, cats, birds (1 second) m...",1
2,2,"Pat: (3 seconds) Um, the picture is of a kitch...","Pat: Um (1 second) purple, pale, placid (1 sec...","Pat: Cow, bull, ewe, ram, chicken, goose, um (...",1
3,3,"Pat: A mother presumably, or a fe, an adult fe...",Pat: Plank (1 second) pool (1 second) swimming...,Pat: Um (1 second) impala (1 second) er cheeta...,1
4,4,Pat: ‘50s style er scene of domestic um confus...,"Pat: It’s er pillock, er post box, er Pyracant...","Pat: Dog, cat, giraffe, wallaby, kangaroo, tor...",1


In [5]:
df.tail()

,recordID,Transcript_CTD,Transcript_PFT,Transcript_SFT,Class_label
770,770,Pat: The sink is overflowing; the woman doing ...,"Pat: Um, precise, prescient, er procrastinatio...","Pat: Um, well (buzzer sounds) er cat, dog, rab...",0
771,771,Pat: I see a scene of absolute chaos in this p...,"Pat: (Buzzer sounds) Picture, plate, palm, pho...","Pat: Cat, lion, tiger (buzzer sounds) oh. Cat,...",0
772,772,Pat: Little boy falling off a chair whilst pas...,"Pat: Countries beginning with P: Paraguay, Por...","Pat: Horse, dog, cat, pig, hen (1 second) walr...",0
773,773,"Pat: (3 seconds) Er, little boy stood on a sto...","Pat: Phew, phew, phew (7 seconds) Phidi, Phila...","Pat: Pig, cat, dog (buzzer sounds) pig, cat, d...",1
774,774,"Pat: (3 seconds) OK, well there’s a boy stood ...","Pat: (3 seconds) Er (2 seconds) paternity, pet...","Pat: Ooh. Armadillo, antelope, bear, buffalo, ...",0


In [6]:
df.shape

(775, 5)

In [7]:
df["label"] = df["Class_label"]

print(df['label'].value_counts(normalize=True))

label
1    0.535484
0    0.464516
Name: proportion, dtype: float64


## Enhanced Text Data Analysis

Comprehensive analysis of transcript data including text statistics, word clouds, and linguistic patterns.

In [12]:
# Interactive Text Analysis Visualizations
print("\nInteractive Text Analysis")

# Prepare data for interactive visualization
text_analysis_data = []

for idx, row in df.iterrows():
    for col in transcript_columns:
        if pd.notna(row[col]):
            text = str(row[col])
            task_type = col.replace('Transcript_', '')
            
            text_analysis_data.append({
                'Task_Type': task_type,
                'Class': 'Dementia' if row['label'] == 1 else 'No Dementia',
                'Word_Count': len(text.split()),
                'Character_Count': len(text),
                'Sentence_Count': len(text.split('.')),
                'Average_Word_Length': np.mean([len(word) for word in text.split()]) if text.split() else 0,
                'Sample_ID': idx
            })

text_df = pd.DataFrame(text_analysis_data)

# Interactive scatter plot
fig_scatter = px.scatter(text_df, 
                        x='Word_Count', 
                        y='Average_Word_Length',
                        color='Class',
                        facet_col='Task_Type',
                        title='Text Characteristics by Task Type and Class',
                        labels={'Word_Count': 'Number of Words',
                               'Average_Word_Length': 'Average Word Length'},
                        hover_data=['Character_Count', 'Sentence_Count'])

fig_scatter.update_layout(height=500)
fig_scatter.show()

# Interactive box plots for different metrics
metrics = ['Word_Count', 'Character_Count', 'Sentence_Count', 'Average_Word_Length']
fig_box = make_subplots(
    rows=2, cols=2,
    subplot_titles=metrics,
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

colors = {'No Dementia': 'blue', 'Dementia': 'red'}

for i, metric in enumerate(metrics):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    for class_label in ['No Dementia', 'Dementia']:
        class_data = text_df[text_df['Class'] == class_label]
        
        fig_box.add_trace(
            go.Box(
                y=class_data[metric],
                x=class_data['Task_Type'],
                name=class_label,
                marker_color=colors[class_label],
                showlegend=(i == 0)  # Only show legend for first subplot
            ),
            row=row, col=col
        )

fig_box.update_layout(
    title='Text Metrics Distribution by Class and Task Type',
    height=700
)
fig_box.show()



Interactive Text Analysis


In [13]:
# Enhanced text cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"\s+", " ", text)  # Preserve punctuation
    return text.strip()

In [14]:
# Apply cleaning to text column
df["Transcript_CTD"] = df["Transcript_CTD"].fillna("").apply(clean_text)
df["Transcript_PFT"] = df["Transcript_PFT"].fillna("").apply(clean_text)
df["Transcript_SFT"] = df["Transcript_SFT"].fillna("").apply(clean_text)


In [15]:
# Combine the three text columns
df["combined_text"] = (
    df["Transcript_CTD"] + " " + df["Transcript_PFT"] + " " + df["Transcript_SFT"]   
)

In [16]:
# Prepare train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["combined_text"], df["label"], test_size=0.2, random_state=42
)


In [17]:
# Ensure they are strings
train_texts = train_texts.astype(str)
test_texts = test_texts.astype(str)

In [18]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(
    list(train_texts), truncation=True, padding=True, max_length=256
)
test_encodings = tokenizer(
    list(test_texts), truncation=True, padding=True, max_length=256
)

In [19]:
# Dataset Wrapper
class DatasetWrapper(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [20]:
train_dataset = DatasetWrapper(train_encodings, list(train_labels))
test_dataset = DatasetWrapper(test_encodings, list(test_labels))

In [21]:
# Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Updated training arguments to optimize for the entire dataset
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,  # Minimum for meaningful learning
    warmup_ratio=0.1,  # 10% of total steps instead of fixed 200
    per_device_train_batch_size=8,  # Increase if GPU memory allows
    gradient_accumulation_steps=2,
)

In [25]:
# Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro')
    }


In [26]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:
# Train
trainer.train()

# Evaluate
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")
print(f"Loss: {results['eval_loss']}")


## Enhanced Model Performance Analysis

Comprehensive analysis of BERT model training and evaluation with detailed visualizations.

In [ ]:
# Enhanced Model Evaluation and Analysis
print("Creating comprehensive model evaluation analysis...")

# Get detailed predictions for analysis
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_pred_proba = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()
y_true = np.array(test_labels)

# 1. Comprehensive Performance Metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

print("\n1. Detailed Performance Analysis")

# Calculate additional metrics
from sklearn.metrics import precision_recall_curve, average_precision_score

# Binary classification metrics
precision_recall_report = classification_report(y_true, y_pred, output_dict=True)
conf_matrix = confusion_matrix(y_true, y_pred)

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Confusion Matrix with detailed annotations
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, 
                             display_labels=['No Dementia', 'Dementia'])
disp.plot(ax=axes[0, 0], cmap='Blues', values_format='d')
axes[0, 0].set_title('Confusion Matrix')

# Normalized Confusion Matrix
conf_matrix_norm = confusion_matrix(y_true, y_pred, normalize='true')
sns.heatmap(conf_matrix_norm, annot=True, fmt='.2f', cmap='Blues',
           xticklabels=['No Dementia', 'Dementia'],
           yticklabels=['No Dementia', 'Dementia'], ax=axes[0, 1])
axes[0, 1].set_title('Normalized Confusion Matrix')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('True')

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)

axes[0, 2].plot(fpr, tpr, color='darkorange', lw=2, 
               label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[0, 2].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[0, 2].set_xlim([0.0, 1.0])
axes[0, 2].set_ylim([0.0, 1.05])
axes[0, 2].set_xlabel('False Positive Rate')
axes[0, 2].set_ylabel('True Positive Rate')
axes[0, 2].set_title('Receiver Operating Characteristic (ROC)')
axes[0, 2].legend(loc="lower right")
axes[0, 2].grid(True, alpha=0.3)

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_true, y_pred_proba[:, 1])
pr_auc = average_precision_score(y_true, y_pred_proba[:, 1])

axes[1, 0].plot(recall, precision, color='blue', lw=2,
               label=f'PR curve (AUC = {pr_auc:.3f})')
axes[1, 0].set_xlabel('Recall')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Precision-Recall Curve')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Prediction Confidence Distribution
confidence_scores = np.max(y_pred_proba, axis=1)
axes[1, 1].hist(confidence_scores, bins=20, alpha=0.7, edgecolor='black')
axes[1, 1].axvline(np.mean(confidence_scores), color='red', linestyle='--', 
                  label=f'Mean: {np.mean(confidence_scores):.3f}')
axes[1, 1].set_xlabel('Prediction Confidence')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Prediction Confidence Distribution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Class-wise Performance Metrics
metrics = ['precision', 'recall', 'f1-score']
classes = ['No Dementia', 'Dementia']
class_metrics = {
    'No Dementia': [precision_recall_report['0'][metric] for metric in metrics],
    'Dementia': [precision_recall_report['1'][metric] for metric in metrics]
}

x_pos = np.arange(len(metrics))
width = 0.35

axes[1, 2].bar(x_pos - width/2, class_metrics['No Dementia'], width, 
              label='No Dementia', alpha=0.8)
axes[1, 2].bar(x_pos + width/2, class_metrics['Dementia'], width,
              label='Dementia', alpha=0.8)

axes[1, 2].set_xlabel('Metrics')
axes[1, 2].set_ylabel('Score')
axes[1, 2].set_title('Class-wise Performance Metrics')
axes[1, 2].set_xticks(x_pos)
axes[1, 2].set_xticklabels([m.title() for m in metrics])
axes[1, 2].legend()
axes[1, 2].set_ylim(0, 1.1)

# Add value labels on bars
for i, metric in enumerate(metrics):
    axes[1, 2].text(i - width/2, class_metrics['No Dementia'][i] + 0.01, 
                   f'{class_metrics["No Dementia"][i]:.3f}',
                   ha='center', va='bottom', fontsize=9)
    axes[1, 2].text(i + width/2, class_metrics['Dementia'][i] + 0.01,
                   f'{class_metrics["Dementia"][i]:.3f}',
                   ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n✓ Comprehensive model evaluation completed")

In [ ]:
# 2. Detailed Prediction Analysis
print("\n2. Prediction Analysis and Model Insights")

# Analyze correct vs incorrect predictions
correct_predictions = (y_true == y_pred)
correct_confidence = confidence_scores[correct_predictions]
incorrect_confidence = confidence_scores[~correct_predictions]

# Create analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Confidence comparison: Correct vs Incorrect
axes[0, 0].hist([correct_confidence, incorrect_confidence], bins=15, 
               alpha=0.7, label=['Correct', 'Incorrect'], color=['green', 'red'])
axes[0, 0].set_xlabel('Prediction Confidence')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Confidence Distribution: Correct vs Incorrect Predictions')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Confidence by True Class
confidence_by_class = []
for class_val in [0, 1]:
    class_mask = y_true == class_val
    class_confidences = confidence_scores[class_mask]
    confidence_by_class.append(class_confidences)

axes[0, 1].boxplot(confidence_by_class, labels=['No Dementia', 'Dementia'])
axes[0, 1].set_xlabel('True Class')
axes[0, 1].set_ylabel('Prediction Confidence')
axes[0, 1].set_title('Confidence Distribution by True Class')
axes[0, 1].grid(True, alpha=0.3)

# Error Analysis by Confidence Threshold
thresholds = np.arange(0.5, 1.0, 0.05)
accuracies_at_threshold = []
coverage_at_threshold = []

for threshold in thresholds:
    high_conf_mask = confidence_scores >= threshold
    if np.sum(high_conf_mask) > 0:
        acc = accuracy_score(y_true[high_conf_mask], y_pred[high_conf_mask])
        coverage = np.sum(high_conf_mask) / len(y_true)
    else:
        acc = 0
        coverage = 0
    
    accuracies_at_threshold.append(acc)
    coverage_at_threshold.append(coverage)

axes[1, 0].plot(thresholds, accuracies_at_threshold, 'bo-', label='Accuracy')
axes[1, 0].plot(thresholds, coverage_at_threshold, 'ro-', label='Coverage')
axes[1, 0].set_xlabel('Confidence Threshold')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Accuracy vs Coverage by Confidence Threshold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Class Probability Distribution
prob_class_0 = y_pred_proba[:, 0]
prob_class_1 = y_pred_proba[:, 1]

# Separate by true class
true_class_0_mask = y_true == 0
true_class_1_mask = y_true == 1

axes[1, 1].scatter(prob_class_0[true_class_0_mask], prob_class_1[true_class_0_mask], 
                  alpha=0.6, label='True: No Dementia', color='blue', s=30)
axes[1, 1].scatter(prob_class_0[true_class_1_mask], prob_class_1[true_class_1_mask], 
                  alpha=0.6, label='True: Dementia', color='red', s=30)
axes[1, 1].plot([0, 1], [1, 0], 'k--', alpha=0.5, label='Decision Boundary')
axes[1, 1].set_xlabel('P(No Dementia)')
axes[1, 1].set_ylabel('P(Dementia)')
axes[1, 1].set_title('Class Probability Space')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3. Interactive Analysis with Plotly
print("\n3. Interactive Model Analysis")

# Create DataFrame for interactive analysis
analysis_df = pd.DataFrame({
    'True_Class': ['No Dementia' if x == 0 else 'Dementia' for x in y_true],
    'Predicted_Class': ['No Dementia' if x == 0 else 'Dementia' for x in y_pred],
    'Confidence': confidence_scores,
    'Prob_No_Dementia': y_pred_proba[:, 0],
    'Prob_Dementia': y_pred_proba[:, 1],
    'Correct': correct_predictions,
    'Sample_ID': range(len(y_true))
})

# Interactive scatter plot of probabilities
fig_interactive = px.scatter(analysis_df, 
                           x='Prob_No_Dementia', 
                           y='Prob_Dementia',
                           color='True_Class',
                           symbol='Correct',
                           size='Confidence',
                           title='Interactive Model Predictions Analysis',
                           labels={
                               'Prob_No_Dementia': 'P(No Dementia)',
                               'Prob_Dementia': 'P(Dementia)',
                               'True_Class': 'True Class'
                           },
                           hover_data=['Predicted_Class', 'Confidence', 'Sample_ID'])

# Add decision boundary
fig_interactive.add_shape(
    type='line',
    x0=0, y0=1, x1=1, y1=0,
    line=dict(color='black', width=2, dash='dash'),
    name='Decision Boundary'
)

fig_interactive.update_layout(height=600)
fig_interactive.show()

# Summary Statistics
print(f"\nModel Performance Summary:")
print("="*50)
print(f"Overall Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Overall F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")
print(f"Mean Confidence: {np.mean(confidence_scores):.4f}")
print(f"Std Confidence: {np.std(confidence_scores):.4f}")

# Confidence Analysis
high_conf_threshold = 0.8
low_conf_threshold = 0.6
high_conf_count = np.sum(confidence_scores >= high_conf_threshold)
medium_conf_count = np.sum((confidence_scores >= low_conf_threshold) & (confidence_scores < high_conf_threshold))
low_conf_count = np.sum(confidence_scores < low_conf_threshold)

print(f"\nConfidence Analysis:")
print(f"High confidence (≥{high_conf_threshold}): {high_conf_count} ({high_conf_count/len(confidence_scores)*100:.1f}%)")
print(f"Medium confidence ({low_conf_threshold}-{high_conf_threshold}): {medium_conf_count} ({medium_conf_count/len(confidence_scores)*100:.1f}%)")
print(f"Low confidence (<{low_conf_threshold}): {low_conf_count} ({low_conf_count/len(confidence_scores)*100:.1f}%)")

# High confidence accuracy
if high_conf_count > 0:
    high_conf_mask = confidence_scores >= high_conf_threshold
    high_conf_accuracy = accuracy_score(y_true[high_conf_mask], y_pred[high_conf_mask])
    print(f"Accuracy for high confidence predictions: {high_conf_accuracy:.4f}")

print("\n✓ Enhanced prediction analysis completed")